# Imports

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, exp, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [2]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_scaled'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')



# Define transport availability
# Assume pt, walking, cycle always available, with car availability depending on number of cars per household. From the data, 
# people without driving licenses choose driving as their mode of transport (eg. row 28). 
av_drive =  (car_ownership > 0)
av_pt =1
av_walk = 1
av_cycle = 1

variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [3]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [4]:
# # MODEL 3


# # Define driving cost
# cost_driving = cost_driving_ccharge + cost_driving_fuel

# # Define time taken by each mode of transport
# dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

# time_pt = dur_pt
# time_cycling = dur_cycling
# time_walking = dur_walking  
# time_driving = dur_driving

# # Model normalized with asc_walking = 0
# asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
# asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
# asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)


# beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_walking_interact = age_scaled * Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_cycling_interact = age_scaled * Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_pt_interact = age_scaled * Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)

# cost_drive_power = power_series(cost_driving)
# cost_pt_power = power_series(cost_transit)

# #mx_age = df['age'].max()

# v_walking = (time_walking * 
#                 (beta_tt_walking + beta_tt_walking_interact * age_scaled))

# v_cycling = (asc_cycling 
#              + time_cycling * 
#                 (beta_tt_cycling + beta_tt_cycling_interact * age_scaled))

# v_pt = (asc_pt 
#         + time_pt * 
#             (beta_tt_pt + beta_tt_pt_interact * age_scaled) 
#         + beta_cost * cost_pt_power)

# v_driving = (asc_driving 
#              + beta_tt_driving * time_driving 
#              + beta_cost * cost_drive_power)

# V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

# model_3 = loglogit(V, None, travel_mode)

In [5]:
# # MODEL 3


# # Define driving cost
# cost_driving = cost_driving_ccharge + cost_driving_fuel

# # Define time taken by each mode of transport
# dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

# time_pt = dur_pt
# time_cycling = dur_cycling
# time_walking = dur_walking  
# time_driving = dur_driving

# # Model normalized with asc_walking = 0
# asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
# asc_cycling = Beta(name='asc_cycling', value=0, lowerbound=None, upperbound=None, status=0)
# asc_driving = Beta(name='asc_driving', value=0, lowerbound=None, upperbound=None, status=0)

# beta_cost = Beta(name='beta_cost', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_walking = Beta(name='beta_tt_walking', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_walking_interact = Beta(name='beta_tt_walking_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_cycling = Beta(name='beta_tt_cycling', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_cycling_interact = Beta(name='beta_tt_cycling_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_pt = Beta(name='beta_tt_pt', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_pt_interact = Beta(name='beta_tt_pt_interact', value=0, lowerbound=None, upperbound=None, status=0)
# beta_tt_driving = Beta(name='beta_tt_driving', value=0, lowerbound=None, upperbound=None, status=0)

# ell_cost = Beta('lambda_cost', 1, -10, 10, 0)
# boxcox_cost_pt = boxcox(cost_transit, ell_cost)
# boxcox_cost_driving = boxcox(cost_driving, ell_cost)

# mx_age = df['age'].max()

# v_walking = (time_walking * 
#                 (beta_tt_walking + beta_tt_walking_interact * age_scaled))

# v_cycling = (asc_cycling 
#              + time_cycling * 
#                 (beta_tt_cycling + beta_tt_cycling_interact * age_scaled))

# v_pt = (asc_pt 
#         + time_pt * 
#             (beta_tt_pt + beta_tt_pt_interact * age_scaled) 
#         + beta_cost * boxcox_cost_pt)

# v_driving = (asc_driving 
#              + beta_tt_driving * time_driving 
#              + beta_cost * boxcox_cost_driving)

# V = {1: v_walking, 2: v_cycling, 3: v_pt, 4: v_driving}

# model_3 = loglogit(V, None, travel_mode)

## BoxCox

In [6]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0)
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) 
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0) 

LAMDA_COST = Beta('lambda_cost', 1, -10, 10, 0)
bx_cost_pt = boxcox(cost_transit, LAMDA_COST)
bx_cost_driving = boxcox(cost_driving, LAMDA_COST)


# Updated utility functions with age interaction for travel time
V_WALK = dur_walking * (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled)

V_CYCLE = (ASC_CYCLE 
           + dur_cycling * (B_TIME_CYCLE + B_TIME_CYCLE_AGE * age_scaled)
        )

V_PT = (ASC_PT 
        + bx_cost_pt * B_COST
        + dur_pt * (B_TIME_PT + B_TIME_PT_AGE * age_scaled)
        )

V_DRIVE = (ASC_DRIVE 
           + bx_cost_driving * B_COST  
           + dur_driving  * B_TIME_DRIVE
        )

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_3_bx = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

### Results

In [7]:
biogeme_bx = bio.BIOGEME(database, model_3_bx)
biogeme_bx.modelName = 'model_3_bx'

results_m3_bx = biogeme_bx.estimate()

print("Estimation results for Model 3 Box Cox:")
print(results_m3_bx.get_estimated_parameters())

Estimation results for Model 3 Box Cox:
                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE        -4.747089      0.216769   -21.899272  0.000000e+00
ASC_DRIVE        -2.890816      0.212370   -13.612177  0.000000e+00
ASC_PT           -2.794560      0.161046   -17.352544  0.000000e+00
B_COST           -0.471095      0.052628    -8.951369  0.000000e+00
B_TIME_CYCLE     -7.073484      0.514990   -13.735185  0.000000e+00
B_TIME_CYCLE_AGE -0.660598      0.234701    -2.814641  4.883169e-03
B_TIME_DRIVE     -6.235618      0.377891   -16.501113  0.000000e+00
B_TIME_PT        -3.825221      0.245230   -15.598522  0.000000e+00
B_TIME_PT_AGE    -0.401068      0.072667    -5.519287  3.403776e-08
B_TIME_WALK      -9.635689      0.522877   -18.428229  0.000000e+00
B_TIME_WALK_AGE  -0.724167      0.159765    -4.532687  5.823801e-06
lambda_cost       0.310434      0.080067     3.877178  1.056749e-04


In [8]:
print(results_m3_bx.print_general_statistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4195.591
Likelihood ratio test for the init. model:	5471.762
Rho-square for the init. model:	0.395
Rho-square-bar for the init. model:	0.393
Akaike Information Criterion:	8415.181
Bayesian Information Criterion:	8493.388
Final gradient norm:	3.0064E-02
Nbr of threads:	16



### Test against Model 2

In [9]:
folder_path = os.path.join(os.pardir, 'Model_2')
file_path = os.path.join(folder_path, 'Model_2B.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [10]:
loglikehood_m3_bx = results_m3_bx.data.logLike
num_params_m3_bx = results_m3_bx.data.nparam

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m3_bx - loglikehood_m2)

# Degrees of freedom
df = num_params_m3_bx - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 3 Box Cox is significantly better than Model 2.")
else:
    print("No significant improvement in Model 3 Box Cox over Model 2.")

Likelihood Ratio: -2.5002939935329778
Degrees of Freedom: 1
Critical Chi-Square Value (0.05 significance): 3.841458820694124
No significant improvement in Model 3 Box Cox over Model 2.


## Power Series

In [11]:
square_tt_coef = Beta('square_cost_coef', 0, None, None, 0)
cube_tt_coef = Beta('cube_cost_coef', 0, None, None, 0)

def power_series(the_variable: Expression) -> Expression:
    """Generate the expression of a polynomial of degree 3

    :param the_variable: variable of the polynomial
    """
    return (
        the_variable
        + square_tt_coef * the_variable**2
        + cube_tt_coef * the_variable * the_variable**3
    )

In [12]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0)
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) 
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0) 

power_cost_driving = power_series(cost_driving)
poewr_cost_pt = power_series(cost_transit)


# Updated utility functions with age interaction for travel time
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking

V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE * age_scaled) * dur_cycling

V_PT = ASC_PT + B_COST * poewr_cost_pt + (B_TIME_PT + B_TIME_PT_AGE * age_scaled) * dur_pt

V_DRIVE = ASC_DRIVE + B_COST * power_cost_driving + (B_TIME_DRIVE) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model
model_3_power = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

### Results

In [13]:
biogeme_power = bio.BIOGEME(database, model_3_power)
biogeme_power.modelName = 'model_3_power'

results_m3_power = biogeme_power.estimate()

print("Estimation results for Model 3 Power:")
print(results_m3_power.get_estimated_parameters())

Estimation results for Model 3 Power:
                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE        -4.611156      0.199839   -23.074386  0.000000e+00
ASC_DRIVE        -2.148531      0.149940   -14.329227  0.000000e+00
ASC_PT           -2.723111      0.150075   -18.145016  0.000000e+00
B_COST            0.005201      0.000579     8.981102  0.000000e+00
B_TIME_CYCLE     -6.651065      0.495164   -13.432041  0.000000e+00
B_TIME_CYCLE_AGE -0.620826      0.226077    -2.746085  6.031111e-03
B_TIME_DRIVE     -7.163034      0.416919   -17.180862  0.000000e+00
B_TIME_PT        -3.603087      0.246244   -14.632198  0.000000e+00
B_TIME_PT_AGE    -0.388389      0.073952    -5.251905  1.505342e-07
B_TIME_WALK      -9.216627      0.481303   -19.149342  0.000000e+00
B_TIME_WALK_AGE  -0.709445      0.156799    -4.524561  6.052102e-06
cube_cost_coef    0.041638      0.005597     7.439362  1.012523e-13
square_cost_coef -8.355522      0.897526    -9.309509  0.000000e+00


In [14]:
print(results_m3_power.print_general_statistics())

Number of estimated parameters:	13
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4202.201
Likelihood ratio test for the init. model:	5458.541
Rho-square for the init. model:	0.394
Rho-square-bar for the init. model:	0.392
Akaike Information Criterion:	8430.403
Bayesian Information Criterion:	8515.126
Final gradient norm:	1.8226E-01
Nbr of threads:	16



### Test Against Model 2

In [15]:
folder_path = os.path.join(os.pardir, 'Model_2')
file_path = os.path.join(folder_path, 'Model_2B.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [16]:
loglikehood_m3_power = results_m3_power.data.logLike
num_params_m3_power = results_m3_power.data.nparam

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (loglikehood_m3_power - loglikehood_m2)

# Degrees of freedom
df = num_params_m3_power - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, df)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", df)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 3 Power is significantly better than Model 2.")
else:
    print("No significant improvement in Model 3 Power over Model 2.")

Likelihood Ratio: -15.721398968782523
Degrees of Freedom: 2
Critical Chi-Square Value (0.05 significance): 5.991464547107979
No significant improvement in Model 3 Power over Model 2.


# Model Results

In [17]:
# biogeme = bio.BIOGEME(database, model_3)
# biogeme.modelName = 'model_3'

# results_m3 = biogeme.estimate()

# print("Estimation results for Model 3:")
# print(results_m3.get_estimated_parameters())

In [18]:
# print(results_m3.print_general_statistics())

# Testing Against Model 2

In [19]:
# folder_path = os.path.join(os.pardir, 'Model_2')
# file_path = os.path.join(folder_path, 'Model_2B.pickle')

# # open a file, where you stored the pickled data
# file = open(file_path, 'rb')

# # dump information to that file
# results_m2 = pickle.load(file)

# # close the file
# file.close()

In [20]:
# loglikehood_m3 = results_m3.data.logLike
# num_params_m3 = results_m3.data.nparam

# loglikehood_m2 = results_m2.logLike
# num_params_m2 = results_m2.nparam

# # Calculate the LR statistic
# LR = 2 * (loglikehood_m3 - loglikehood_m2)

# # Degrees of freedom
# df = num_params_m3 - num_params_m2

# # Critical value at 0.05 significance level
# critical_value = chi2.ppf(0.95, df)

# print("Likelihood Ratio:", LR)
# print("Degrees of Freedom:", df)
# print("Critical Chi-Square Value (0.05 significance):", critical_value)

# if LR > critical_value:
#     print("Model 3 is significantly better than Model 2.")
# else:
#     print("No significant improvement in Model 3 over Model 2.")

